<a href="https://colab.research.google.com/github/gil-kapel/video_analysis/blob/main/audio_transcriber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install googletrans==3.1.0a
!pip install pytubefix
!pip install langdetect
!pip install pydub
!pip install huggingsound
!pip install transformers --upgrade
!pip install langcodes
!pip install torch torchvision torchaudio
from IPython.display import clear_output
clear_output()

  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
huggingsound 0.1.6 requires torch!=1.12.0,<1.13.0,>=1.7, but you have torch 2.3.1 which is incompatible.


In [5]:
import os
from pytubefix import YouTube
from langdetect import detect, detect_langs
from pydub import AudioSegment
import torch
import langcodes
from huggingsound import SpeechRecognitionModel


def download_youtube_audio(url):
    try:
        yt = YouTube(url)
        video = yt.streams.filter(only_audio=True).first()
        if video is None:
            raise Exception("No audio streams available for this video.")
        out_file = video.download(output_path=".")
        base, ext = os.path.splitext(out_file)
        audio = AudioSegment.from_file(out_file)
        os.remove(out_file)

        new_file = base + '.wav'
        audio.export(new_file, format="wav")
        language = detect_langs(yt.title)
        return new_file, language[0].lang
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None


def transcribe_audio(file_path, language_code):
    """
    Transcribes a .wav audio file into text using a Wav2Vec2 model from Hugging Face.
    Args:
        file_path (str): Path to the .wav audio file.
        language_code (str): Language code (e.g., 'en' for English, 'fr' for French).

    Returns:
        str: The transcribed text.
    """
    full_lang = langcodes.Language.get(language_code).display_name().lower()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model = SpeechRecognitionModel(f"jonatasgrosman/wav2vec2-large-xlsr-53-{full_lang}", device=device)
    transcriptions = model.transcribe([file_path])
    return transcriptions[0]


# youtube_url = input("Enter YouTube URL: ")
youtube_url = "https://www.youtube.com/watch?v=WdOPb8YvxZg"

audio_file, lang_code = download_youtube_audio(youtube_url)
if audio_file is None:
    print("Failed to download audio. Exiting.")

print("Audio downloaded. Transcribing...")
text = transcribe_audio(audio_file, language_code=lang_code)

print("Transcription complete.")
print(text['transcription'])
os.remove(audio_file)




INFO:huggingsound.speech_recognition.model:Loading model...


Audio downloaded. Transcribing...
Using device: cuda


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-persian were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a Be

Transcription complete.
{'transcription': 'جمهوری اسلامی وچهرههای ایانیهایی که خال چس کشور ارفت جلوی اسفارهت شروع کرده شناسایی کردن این خال خطرنا که  تهدید داده میکنه بدای هعملیات تروریستی کریپی داده این حالا کهلیپ شماره دو فاض دوم شناسایی نفرت گره د در چندین کیلی و گروهایه مخت گروهاستون گور چهارم گوره پنجم درست کرد و میگه عملیات برخود با اوامل ایجاد ناامنی برای شهروندان جموری اسلامییران د خارد ات کشو ایام انتخابات ریاسجمرییه موزیک خیلی ترسناکی هم گذاشته بود که من این موزیک و قتع کردم الاه و بهبوزیک قت کرن و میرورو چهرههامن برخی ازین چهرهعارم میشناسم در اروپاوداره تهدیلشون میکنیششنیخب بنین چرار دروادهوسهخانه نگه سوت همه چیز اسمشونو درورده شما ره این ملیشونو برم ساکنه کجاستوو ببی اینجوری اادید ودادخب بدون دونه درعادهماعریاشیرف کرد ان فالهها نام پدر فالا ساکنهونصبت ات دو ستخادمون حسن جفری بوده تویسور عبچها کهل خوبیم حای یهمون تخاقات نمایشی فرمایشیجکنی اسلانیقبللامی جاخسی چخونی اسلامی هستیم ی اعلام کردن نسرات سجارمون و فختنایب قنه جهای هسن جافری بکه من اصل دو ازاریازم باشی دوستی داشمت دیگ

In [42]:
from googletrans import Translator
translator = Translator()
result = translator.translate(text['transcription'][:3181], src=lang_code, dest='en')
print(result.text)


The Islamic Republic of Iran and the faces of the people who have left the country in front of the embassy have started to identify this dangerous mole that threatens the beginning of the terrorist operations. Kurdish and says that the operation was aimed at creating insecurity for the citizens of the Islamic Republic of Iran. During the election days of the President of the Republic of Iran, he also played very scary music. Why are you at the door of the house, keep the whistle of everything, their names and their nationalities, where do you live? Where are you? Announcing the blessings of Nasrat Sajarmon and Fakhtanaib Qanejai Hasen Jafari, I want you to be my main friend, I have been friends with you for years, we don't have anything to do anymore. The white forehead is known by everyone, and my body is not scary. They say that they are serving you at the table. They say that they have seen it. What you are doing is not scary. That is the way you are. The children who go forgery are

In [ ]:
# prompt: build me a huuging face model that gets a long text like an article and summarize it, returns main topics, key insights etc

!pip install transformers

from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_text(text):
  """Summarizes a given text using a pre-trained Hugging Face model.

  Args:
    text: The text to summarize.

  Returns:
    A dictionary containing the summary and key insights.
  """

  summary = summarizer(text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']

  # For key insights, you can use a keyword extraction technique or another summarization model.
  # For simplicity, we'll just return the first few sentences of the summary as "key insights".
  key_insights = " ".join(summary.split('.')[:2]) + "."

  return {
      "summary": summary,
      "key_insights": key_insights
  }

# Example usage:
article_text = """Your long article text goes here..."""
result = summarize_text(article_text)
print("Summary:", result["summary"])
print("\nKey Insights:", result["key_insights"])
